In [ ]:
%pip install pandas
%pip install skillcorner
%pip install git+https://github.com/liamMichaelBailey/skillcorner_analysis_toolkit.git

In [ ]:
import pandas as pd
from skillcorner.client import SkillcornerClient
from skillcorner_analysis_toolkit.skillcorner_plots import plot_bar_chart, plot_scatter, plot_swarm_violin
from skillcorner_analysis_toolkit.skillcorner_normalisations import get_per_30_tip, get_per_100
from skillcorner_analysis_toolkit.constants import *

In [ ]:
USERNAME = ''
PASSWORD = ''
client = SkillcornerClient(username=USERNAME, password=PASSWORD)
df = pd.DataFrame(client.get_in_possession_off_ball_runs(params={'run_type': ','.join(REQUEST_RUN_TYPES),
                                                                 'season': 21,
                                                                 'competition': 1,
                                                                 'count_match__gte': 8,
                                                                 'playing_time__gte': 60,
                                                                 'group_by': 'player,team,competition,group,position',
                                                                 'average_per': 'match'}))

In [ ]:
df.head()

In [ ]:
df['count_runs_per_30_tip'] = get_per_30_tip(df, 'count_runs_per_match')

fig, ax = plot_bar_chart(df=df[df['position'] == 'CF'],
                         x_metric='count_runs_per_30_tip',
                         x_label='Number Of Off-Ball Runs P30 TIP',
                         primary_highlight_group=['Roberto Firmino',
                                            'Darwin Nunez',
                                            'Cody Gakpo'])

fig.savefig('center_forward_run_volume.png', format='png', dpi=300)

In [ ]:
df['count_cross_receiver_runs_per_30_tip'] = get_per_30_tip(df, 'count_cross_receiver_runs_per_match')
df['count_runs_in_behind_per_30_tip'] = get_per_30_tip(df, 'count_runs_in_behind_per_match')

fig, ax = plot_scatter(df=df[df['position'] == 'CF'],
                       x_metric='count_runs_in_behind_per_30_tip',
                       y_metric='count_cross_receiver_runs_per_30_tip',
                       x_label='Number Of Runs In behind P30 TIP',
                       y_label='Number Of Coming Short Runs P30 TIP',
                       x_sd_highlight=.5,
                       y_sd_highlight=.5,
                       primary_highlight_group=['Roberto Firmino',
                                            'Darwin Nunez',
                                            'Cody Gakpo'])

fig.savefig('center_forward_run_profile.png', format='png', dpi=300, transparent=False, facecolor='w')

In [ ]:
df['threat_per_100_cross_receiver_runs'] = get_per_100(df=df,
                                                       metric_per_match='cross_receiver_runs_threat_per_match',
                                                       adjustment_metric_per_match='count_cross_receiver_runs_per_match')

df['threat_per_100_runs_in_behind'] = get_per_100(df=df,
                                                       metric_per_match='runs_in_behind_threat_per_match',
                                                       adjustment_metric_per_match='count_runs_in_behind_per_match')

df['threat_per_100_runs_ahead_of_the_ball'] = get_per_100(df=df,
                                                       metric_per_match='runs_ahead_of_the_ball_threat_per_match',
                                                       adjustment_metric_per_match='count_runs_ahead_of_the_ball_per_match')

melted_df = df.melt(id_vars=["player_id",
                             "player_name",
                             "player_birthdate",
                             "team_id",
                             "team_name",
                             "competition_id",
                             "competition_name",
                             "group",
                             "position",
                             "third",
                             "channel",
                             "minutes_played_per_match",
                             "count_match",
                             "count_match_failed"],
                    var_name="metric",
                    value_name="value")

x_value = 'value'  # This should remain as 'value'.
y_value = 'metric'  # This should remain as 'metric'.

In [ ]:
melted_df.head()

In [ ]:
fig, ax = plot_swarm_violin(df=melted_df[melted_df['position'] == 'CF'],
                      x_metric=x_value,
                      y_metric=y_value,
                      y_groups=['threat_per_100_cross_receiver_runs',
                                'threat_per_100_runs_in_behind',
                                'threat_per_100_runs_ahead_of_the_ball'],
                      x_label='Threat P100 Off-Ball Runs',
                      y_group_labels=['Cross Receiver Runs',
                                      'Runs In Behind',
                                      'Runs Ahead Of The Ball'],
                      primary_highlight_group=['Roberto Firmino',
                                               'Darwin Nunez',
                                               'Cody Gakpo'])

fig.savefig('center_forward_run_threat.png', format='png', dpi=300, transparent=False, facecolor='w')